![](http://kodcu.com/wp/wp-content/uploads/2014/06/mllib.png)

<img src="http://img.scoop.it/_VP0qLV-jYbp2cFF4H4k4jl72eJkfbmt4t8yenImKBVvK0kTmF0xjctABnaLJIm9" style="width: 600px;"/>

In [8]:
from pyspark.mllib import *

![](https://media.giphy.com/media/b7FNjKdGXEFos/giphy.gif)

---
By the end of this session, you should be able to:
----

- Describe Spark's machine learning framework - MLlib
- Fit K-means clustering
- Apply Collaborative Filtering to make recommendations

---
MLlib Overview
---

MLlib is a Spark subproject providing machine learning primitives that are "production" ready.

Started by AMPLab at UC Berkeley

Widespread adoption and support - 80+ contributors from various organization


----
MLlib is not always needed
----

__Remember__: You do not _need_ to use MLlib to do large scale machine learning

Depending on the nature of the problem:

1. Use a large single-node machine learning library (_i.e._ `scikit-learn`) on big EC2
2. Distrubte embarassingly parallel problems as grid search without a framework 

----
MLlib Algorithms
----

- logistic regression and linear support vector machine (SVM)
- classification and regression tree
- random forest and gradient-boosted trees
- recommendation via alternating least squares (ALS)
- clustering via k-means, bisecting k-means, Gaussian mixtures (GMM), and power iteration clustering
- topic modeling via latent Dirichlet allocation (LDA)
- survival analysis via accelerated failure time model
- singular value decomposition (SVD) and QR decomposition
- principal component analysis (PCA)
- linear regression with L1, L2, and elastic-net regularization
- isotonic regression
- multinomial/binomial naive Bayes
- frequent itemset mining via FP-growth and association rules
- sequential pattern mining via PrefixSpan
- summary statistics and hypothesis testing
- feature transformations
- model evaluation and hyper-parameter tuning

[As of 2016-06-14](http://spark.apache.org/mllib/)

---
Points to Ponder
---

<details><summary>
What algorithms are not in Spark? Why do you think that is?
</summary>
Deep Learning. [They are working on it](http://arxiv.org/abs/1511.06051)
</details>

---
Statistics
---

<img src="https://camo.githubusercontent.com/7fdcabe25caf35b9001e5ed1ec9f100d16d77d2d/687474703a2f2f7777772e6d61746866756e6e792e636f6d2f696d616765732f6d6174686a6f6b652d686168612d68756d6f722d6d6174682d6d656d652d6a6f6b652d7069632d6d6174686d656d652d66756e6e79706963732d70756e2d7374616e64617264646576696174696f6e2d737461746973746963732d6e6f726d2d6c6f76652e6a7067" style="width: 400px;"/>

In [9]:
import numpy as np

from pyspark.mllib.stat import Statistics

In [3]:
# Let's look at the what is available...
Statistics.

SyntaxError: invalid syntax (<ipython-input-3-12e05460d4e2>, line 2)

In [10]:
# Create a RDD of Vectors
mat = sc.parallelize([np.array([1.0, 10.0, 100.0]), np.array([2.0, 20.0, 200.0]), np.array([3.0, 30.0, 300.0])])  
mat.collect()

[array([   1.,   10.,  100.]),
 array([   2.,   20.,  200.]),
 array([   3.,   30.,  300.])]

In [11]:
# Compute column summary statistics.
summary = Statistics.colStats(mat)

In [12]:
# Let's look at the what is available...
summary.

In [13]:
print("The mean of each column: {}".format(summary.mean()))
print("The mean of each clumn: {}".format(summary.variance()))

The mean of each column: [   2.   20.  200.]
The mean of each clumn: [  1.00000000e+00   1.00000000e+02   1.00000000e+04]


---
Check for understanding
---
<p><details><summary>Which common statistics are missing from this list? Why?</summary>
Median and other quantiles are missing.  
<br>
Why? Because they are non-associative, thus harder to parrelleize
</details></p>

[Source](https://people.apache.org/~pwendell/spark-nightly/spark-master-docs/latest/mllib-statistics.html)

<br>
<br> 
<br>

----
![](images/kmeans.png)

![](images/clusters.png)

![](images/choose.png)

![](images/adjust.png)

![](images/spark_kmeans.png)

K-means in Spark 2.0
---

[Databbrick K-means demo](https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/6058142077065523/2030838328691810/4338926410488997/latest.html)

In [ ]:
# from pyspark.ml.linalg import Vectors
# from pyspark.ml.clustering import KMeans

In [ ]:
# data = [(Vectors.dense([0.0, 0.0]),), (Vectors.dense([1.0, 1.0]),),
#         (Vectors.dense([9.0, 8.0]),), (Vectors.dense([8.0, 9.0]),)]
# df = spark.createDataFrame(data, ["features"])

In [ ]:
# kmeans = KMeans(k=2, seed=1)
# model = kmeans.fit(df)

In [ ]:
# centers = model.clusterCenters()
# centers

[Source]( https://people.apache.org/~pwendell/spark-nightly/spark-master-docs/latest/api/python/pyspark.ml.html?highlight=kmeans#pyspark.ml.clustering.KMeans )

---
Check for understanding
---

<details><summary>
Why is K-means in Spark's MLlib?
</summary>
K-means requires a complete pass over the data every iteration. Spark is relatively efficient for that specification.
</details>

<br>
<br> 
<br>

----
Recommendations via matrix completion
----

![](images/recommend.png)

Collaborative Filtering
---

Goal: predict users’ movie ratings based on past ratings of other movies

$$
R = \left( \begin{array}{ccccccc}
1 & ? & ? & 4 & 5 & ? & 3 \\
? & ? & 3 & 5 & ? & ? & 3 \\
5 & ? & 5 & ? & ? & ? & 1 \\
4 & ? & ? & ? & ? & 2 & ?\end{array} \right)
$$


<details><summary>
What are the challenges of large scale Collaborative Filtering?
</summary>
Challenges: <br>
- Defining similarity that scales <br>
- Dimensionality (Millions of Users / Items) <br>
- Sparsity, most users has not seen most items <br>
<br>
</details>

![](images/solution.png)

![Matrix Factorization](https://databricks-training.s3.amazonaws.com/img/matrix_factorization.png)

## Alternating Least Squares  
![ALS](https://databricks.com/wp-content/uploads/2014/07/als-illustration.png)  
1. Start with random $U_1$, $V_1$
2. Solve for $U_2$ to minimize $||R – U_2V_1^T||$ 
3. Solve for $V_2$ to minimize $||R – U_2V_2^T||$ 
4. Repeat until convergence

----
ALS on Spark
---

Cache 2 copies of $R$ in memory, one partitioned by rows and one by columns 

Keep $U~\&~V$ partitioned in corresponding way 

Operate on blocks to lower communication


ALS in Spark 2.0
---

[Databbrick ALS demo](https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/6058142077065523/2030838328691818/4338926410488997/latest.html)

In [ ]:
# from pyspark.mllib.recommendation import ALS

In [ ]:
# df = spark.createDataFrame([(0, 1, 4.0), (1, 1, 5.0), (1, 2, 4.0), (2, 1, 1.0), (2, 2, 5.0)],
#                            ["user", "item", "rating"])
# display(df)

In [ ]:
# als = ALS() 
# model = als.train(df, rank=10)

In [ ]:
# user = 0
# item = 2
# model.predict(user, item)

[Source](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#module-pyspark.mllib.recommendation)

---
ALS vs. SVD for recommendation engines
---

Alternating least squares (ALS) is flexible but less precise.  "Approximate" means minimizing some squared-error difference with the input A, but, here you can customize exactly what is considered in the loss function. For example you can ignore missing values (crucial) or weight different values differently.

Singular value decomposition (SVD) is a decomposition that gives more guarantees about its factorization. The SVD is relatively more computationally expensive and harder to parallelize. There is also not a good way to deal with missing values or weighting; you need to assume that in your sparse input, missing values are equal to a mean value 0. 

[Source](https://www.quora.com/What-is-the-difference-between-SVD-and-matrix-factorization-in-context-of-recommendation-engine)

---
Summary
----

- MLlib is Spark's machine learning framework. 
- A bunch of super smart people are porting even more algorithms to work in distributed, lazy-exectution way.+987
- MLlib has the greatest hits of machine learning:
    - K-means for clustering
    - ALS for Collaborative Filtering

<br>
<br>
----

---
Bonus Materials
---

---
MLlib Data Types
---

[Docs](http://spark.apache.org/docs/latest/mllib-data-types.html)

`Local vector`: can be dense or sparse

A dense vector is backed by a double array representing its entry values.

While a sparse vector is backed by two parallel arrays: indices and values. 

For example, a vector (1.0, 0.0, 3.0) can be represented in 

| Dense | Sparse |  
|:-------:|:------:|
| [1.0, 0.0, 3.0]  | (3, [0, 2], [1.0, 3.0]) |


In [6]:
from pyspark.mllib.linalg import Vectors

# Create a SparseVector.
sv1 = Vectors.sparse(3, [0, 2], [1.0, 3.0])

In [16]:
sv1

SparseVector(3, {0: 1.0, 2: 3.0})

`LabeledPoint`

A labeled point is a local vector, either dense or sparse, associated with a label/response for supervised learning algorithms

In [9]:
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint

# Create a labeled point with a positive label and a dense feature vector.
pos = LabeledPoint(1.0, [1.0, 0.0, 3.0])

# Create a labeled point with a negative label and a sparse feature vector.
neg = LabeledPoint(0.0, SparseVector(3, [0, 2], [1.0, 3.0]))

Local matrix

Take a _wild guess_...

> A local matrix has integer-typed row and column indices and double-typed values, stored on a single machine. 

Distributed matrix

> A distributed matrix has long-typed row and column indices and double-typed values, stored distributively in one or more RDDs. 

It is very important to choose the right format to store large and distributed matrices. Converting a distributed matrix to a different format may require a global shuffle, which is quite expensive. Three types of distributed matrices have been implemented so far.

In [12]:
from pyspark.mllib.linalg.distributed import RowMatrix

# Create an RDD of vectors.
rows = sc.parallelize([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])

# Create a RowMatrix from an RDD of vectors.
mat = RowMatrix(rows)

# Get its size.
m = mat.numRows()  # 4
n = mat.numCols()  # 3

# Get the rows as an RDD of vectors again.
rowsRDD = mat.rows

In [15]:
rowsRDD.<tab>

---
Check for understanding
---
<details><summary>
Which data type would you use for random forests?
</summary>
`LabeledPoint` <br>
<br>
Random forests is a supervised learning algorithm. It requires labels in order to classify.
</details>

<br>
<br> 
<br>

----